In [1]:
pip install ultralytics
# The "ultralytics" package is a collection of deep learning tools, including implementation of the YOLOv8 object detection algorithms, as well as other computer vision models.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 25.4 MB/s eta 0:00:00


In [9]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')
## connecting to the google drive

Mounted at /content/drive


In [4]:
# As per yolov8 documentation, directiory names:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [10]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images
    

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    ## calculates the number of images that should be moved from the "path" directory to the "test" directory
    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    
    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]): #tqdm() function is used to create a progress bar for the loop.
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')
        
    

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")
    
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')
            
        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")
    
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")
    


    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")
    
    print("------ TASK COMPLETED -------")



### Function Call
train_test_split('/content/drive/MyDrive/yoloV8/dataset/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 64 images---


  0%|          | 0/52 [00:00<?, ?it/s]

------ Training data created with 80% split 52 images -------


  0%|          | 0/12 [00:00<?, ?it/s]

------ Testing data created with a total of 12 images ----------
------ TASK COMPLETED -------


In [11]:
import ultralytics
ultralytics.checks()
## checks the system requirements and dependencies for the package to function properly    

Ultralytics YOLOv8.0.48 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 25.9/107.7 GB disk)


# Training


In [13]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/yoloV8/dataset.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/yoloV8/training_result name= football

Ultralytics YOLOv8.0.48 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/yoloV8/dataset.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yoloV8/training_result, name=football, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, min_memory=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=Fal

# Test

In [21]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yoloV8/training_result/football/weights/best.pt conf=0.35 source=/content/drive/MyDrive/yoloV8/test_images save=True

Ultralytics YOLOv8.0.48 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/10 /content/drive/MyDrive/yoloV8/test_images/1 (1).jpg: 448x640 7 players, 389.0ms
image 2/10 /content/drive/MyDrive/yoloV8/test_images/1 (4).jpg: 448x640 7 players, 374.8ms
image 3/10 /content/drive/MyDrive/yoloV8/test_images/1 (5).jpg: 448x640 4 players, 377.1ms
image 4/10 /content/drive/MyDrive/yoloV8/test_images/1 (7).jpg: 544x640 10 players, 453.6ms
image 5/10 /content/drive/MyDrive/yoloV8/test_images/1 (8).jpg: 448x640 10 players, 375.9ms
image 6/10 /content/drive/MyDrive/yoloV8/test_images/1 (9).jpg: 448x640 (no detections), 372.3ms
image 7/10 /content/drive/MyDrive/yoloV8/test_images/child-613199_960_720.jpg: 544x640 1 player, 686.3ms
image 8/10 /content/drive/MyDrive/yoloV8/test_images/football-1419954__480.jpg: 448x640 (no detections), 564.8ms
image 9/10 /content/drive/MyDrive/yoloV8/test_images/photo-1551958219-acbc608c6377.

In [22]:
# to save the predected result in drive from ram
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yoloV8/output

In [23]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yoloV8/training_result/football/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yoloV8/video save=True

Streaming output truncated to the last 5000 lines.
video 1/1 (807/5804) /content/drive/MyDrive/yoloV8/video/Football in Ultra HD (2160p 4k).mp4: 384x640 3 players, 327.0ms
video 1/1 (808/5804) /content/drive/MyDrive/yoloV8/video/Football in Ultra HD (2160p 4k).mp4: 384x640 4 players, 325.1ms
video 1/1 (809/5804) /content/drive/MyDrive/yoloV8/video/Football in Ultra HD (2160p 4k).mp4: 384x640 4 players, 324.9ms
video 1/1 (810/5804) /content/drive/MyDrive/yoloV8/video/Football in Ultra HD (2160p 4k).mp4: 384x640 3 players, 321.8ms
video 1/1 (811/5804) /content/drive/MyDrive/yoloV8/video/Football in Ultra HD (2160p 4k).mp4: 384x640 2 players, 329.6ms
video 1/1 (812/5804) /content/drive/MyDrive/yoloV8/video/Football in Ultra HD (2160p 4k).mp4: 384x640 3 players, 329.7ms
video 1/1 (813/5804) /content/drive/MyDrive/yoloV8/video/Football in Ultra HD (2160p 4k).mp4: 384x640 3 players, 325.6ms
video 1/1 (814/5804) /content/drive/MyDrive/yoloV8/video/Football in Ultra HD (2160p 4k).mp4: 384x640 

In [25]:
# to save the predected result in drive from ram
!cp -r /content/runs/detect/predict4 /content/drive/MyDrive/yoloV8/output